## Case Study 3 (CS3) - Sill Complex, Northwestern Australia
### Model C - Built using our adaptation of the ODSIM (Henrion et al. 2008,2010) and structural frames (Grose et al. 2021).

- Considers a sample of approximately a 0.1% of the original dataset (Koepping et al. 2022). These points were classified depending on their location as top, base and lateral contacts.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loop library
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer 

In [ ]:
from datetime import datetime

In [ ]:
lower_extent = [189000, 7775000, -12000]
upper_extent = [196350, 7790050, -8500]

#### 1. Load data

##### Input DataFrame description
> feature_name = name of the geological feature to be modelled. Faults 3 and 6 are the marginal faults that facilitated the emplacement of the inward-dipping outer sills

> (X, Y, Z) = data points location

> coord = structural frame coordinate

> val = values of the scalar field for interpolation of geological features

> (gx, gy, gz) = gradients of structural frame scalar fields

> intrusion_contact_type = roof/top or floor/base

> intrusion_side = TRUE if lateral contact, blank if not

> intrusion_anisotropy = name of geological feature in contact with intrusion (i.e. fault or stratigraphic unit)

In [ ]:
model_data = pd.read_csv('S3 Sill - Model C.csv')
sill_data = model_data[model_data['feature_name'] == 'S3'].copy()

model_data.head()

#### 2. Define conceptual model function to constraint lateral and vertical extent

These conceptual models are used to constraint the simulation of the intrusion lateral and vertical extent.

In [ ]:
def ellipse_function(
    lateral_contact_data = pd.DataFrame() , model = False, minP=None, maxP=None, minS=None, maxS=None
):
    
    if lateral_contact_data.empty:
        return model, minP, maxP, minS, maxS
    else:
        if minP == None:
            minP = lateral_contact_data["coord1"].min()
        if maxP == None:
            maxP = lateral_contact_data["coord1"].max()
        if minS == None:
            minS = lateral_contact_data["coord2"].abs().min()
        if maxS == None:
            maxS = lateral_contact_data["coord2"].max()

        a = (maxP - minP) / 2
        b = (maxS - minS) / 2

        po = minP + (maxP - minP) / 2

        p_locations = lateral_contact_data.loc[:, "coord1"].copy().to_numpy()

        s = np.zeros([len(p_locations), 2])

        s[np.logical_and(p_locations>minP, p_locations<maxP),0] =  b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 
        s[np.logical_and(p_locations>minP, p_locations<maxP),1] =  -b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 

        return s

def constant_function(
    othercontact_data = pd.DataFrame(),
    mean_growth=None, 
    minP=None, 
    maxP=None, 
    minS=None, 
    maxS=None, 
    vertex=None):
    
    if othercontact_data.empty:
        return mean_growth
    
    if mean_growth == None:
        mean_growth = othercontact_data.loc[:,'coord0'].mean()
        
    data_ps = np.array([othercontact_data.loc[:,'coord1'], othercontact_data.loc[:,'coord2']]).T
    
    conceptual_growth = np.ones([len(data_ps),2]) * mean_growth
    
    return conceptual_growth

#### 3. Create geological model

In [ ]:
datetime.now().isoformat(timespec='seconds')   

In [ ]:
# Create Geological Model

model = GeologicalModel(lower_extent,upper_extent)
displacement = 0
# model.nsteps = [40,40,150]
model.data = model_data

fault3 = model.create_and_add_fault('fault_3',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)
fault6 = model.create_and_add_fault('fault_6',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)
fault7 = model.create_and_add_fault('fault_7',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)

conformable_feature = model.create_and_add_foliation('stratigraphy', nelements = 3000, solver = 'lu', interpolatortype = 'FDI')

intrusion_frame_parameters = {'type' : 'interpolated' , 
                              'contact' :'floor',
                              'contact_anisotropies':[conformable_feature]}

print(datetime.now().isoformat(timespec='seconds'))

S3_Sill = model.create_and_add_intrusion('S3', intrusion_frame_name = 'S3 frame',
                                         intrusion_lateral_extent_model = ellipse_function,
                                         intrusion_vertical_extent_model = constant_function,
                                         intrusion_frame_parameters = intrusion_frame_parameters,
                                        interpolatortype = 'FDI')

print(datetime.now().isoformat(timespec='seconds'))

In [ ]:
nn = 100 # improve model resolution by changing this value
model.nsteps = [nn,nn,nn*2]

In [ ]:
from datetime import datetime
datetime.now().isoformat(timespec='seconds')   

In [ ]:
viewer = LavaVuModelViewer(model, background='white')

viewer.add_points(sill_data.loc[:,['X','Y','Z']].to_numpy(), name = 'S3 sill data', pointsize = 5)

# add stratigraphy
viewer.add_isosurface(conformable_feature, isovalue = 0, colour = 'grey')

# add faults
viewer.add_isosurface(fault3[0], isovalue = 0, colour = 'blue')
viewer.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer.add_isosurface(fault7[0], isovalue = 0, colour = 'blue')

# add intrusion
viewer.add_isosurface(S3_Sill, isovalue = 0, colour = 'red')
viewer.ymin = 0
viewer.xmin = 0
viewer.ymax = 1
viewer.xmax = 1
viewer.rotate([-46.267, -38.008, -17.659])
viewer.interactive()

In [ ]:
datetime.now().isoformat(timespec='seconds')   